<a href="https://colab.research.google.com/github/blue-create/langlens/blob/main/models/rule_based/rule_based_labelling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Imports

In [1]:

import tqdm as tqdm
from collections import Counter
import os
import pandas as pd
import re
from ast import literal_eval
import statistics
import matplotlib.pyplot as plt
import re
import pandas as pd

folder_path = "filtered/filtered_06_15 (only kw)"

### Data

In [3]:
# connect with google drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
# redirect the working directory of this script to the data folder
%cd /content/drive/MyDrive/Work/Frontline/data/
#%cd /content/drive/MyDrive/data/

/content/drive/.shortcut-targets-by-id/1WfnZsqpG1r110J63sMbfS5TpsDOkveiV/data


In [5]:


dfs = []

# loop through files
for filename in os.listdir(folder_path):
    # if csv file, load and add to dfs
    if filename.endswith(".csv"):
        file_path = os.path.join(folder_path, filename)
        # import csv with text as list object
        df = pd.read_csv(file_path, index_col=0, converters={"text":literal_eval})
        dfs.append(df)
# combine files in df
df_filtered = pd.concat(dfs, ignore_index=True)


In [ ]:
df=pd.read_csv("filtered/filtered_06_16.csv",converters={"text":literal_eval})

In [34]:
df_test=df.iloc[:500,:]

### 1. Sensationalism

In [42]:
from scripts import annotations

Looking for "sensationalist speech" in previously annotated Paragraphs

In [45]:
ann=pd.read_csv("annotated/230613_all_annotations.csv", index_col=0,converters={"annotations":literal_eval})
ann.loc[:,"annotations"]=ann.apply(annotations.ground_truth_filter,min_coannotation=1,min_similarity=-1, similarity="dice",axis=1)
sensationalist_texts=ann[ann.annotations=={"Sensationalist"}][["text"]]
sensationalist_texts

In [59]:
sens =["Ehetragödie", "verschmähter? Liebhaber(in)?", "rachsüchtige Hexe", "Eifersuchtstat", "Beziehungsdramas?", "Rosenkrieg",
       ##new
       "Frauenschläger","tödliches?m? Drama","tränenerstickter? Stimme","blutüberströmt","tödlichen? Eskalation"]

In [60]:
def label_sensationalism(text):
  for pattern in sens:
    if len(re.findall(pattern=pattern,string=text,flags=re.IGNORECASE))>0:
      return "Sensationalist"
  return ""

In [61]:
df_test=df_test.explode("text")

In [62]:
# apply labelling to test set
df_test["label"]=df_test.text.apply(label_sensationalism)

In [66]:
df_test[df_test["label"]=="Sensationalist"][["text"]].head()

,text
16,Wer so viel Geld für Action und Digitaleffekte...
83,"Die Darstellerin hatte keine Ahnung, was auf s..."
110,Am Sonntagmorgen kam es in Strasburg (Neubrand...
124,Kai Schumann wurde am 28. Juli 1976 in Dresden...
200,Nur 15 Monate hielt die Ehe. Der Rosenkrieg da...


### 2. Statistics

In [103]:
stats=["\d*([\.,]\d*)?[ ](Prozent|%)","\d+ (Fälle|Opfer|Frauen|Kinder|Personen|Betroffene|Morde|Plätze)\W", "Bundeskriminalamt","Kriminalstatistik","statistisches Amt","\Wbka\W"]

In [67]:
def label_statistics(text):
  for pattern in stats:
    if len(re.findall(pattern=pattern,string=text,flags=re.IGNORECASE))>0:
      return "Statistic"
  return ""

In [104]:
# apply labelling to test set
df_test["label"]=df_test.text.apply(label_statistics)

In [ ]:
df_test[df_test["label"]=="Statistic"][["text"]].head()

### 3. Support Services

In [106]:
support=["[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Za-z]{2,}",'\(?\d{4,5}\)?[/\s]*\d{1,5}\s*\d{1,5}',"(www\.)\S+\.\S+"]

In [107]:
def label_support(text):
  for pattern in support:
    if len(re.findall(pattern=pattern,string=text,flags=re.IGNORECASE))>0:
      return "Support Service"
  return ""

In [ ]:
# apply labelling to test set
df_test["label"]=df_test.text.apply(label_support)
df_test[df_test["label"]=="Support Service"][["text"]].head()

### 4. Origin/ Nationality

In [117]:
def label_origin(text):
  for pattern in nationality:
    if len(re.findall(pattern=pattern,string=text,flags=re.IGNORECASE))>0:
      return "Nationality"
  return ""

In [ ]:
nationality=["Afghane", "Afghanin","Albaner", "Albanerin","Algerier", "Algerierin","Argentinier", "Argentinierin","Armenier", "Armenierin","Aserbaidschaner", "Aserbaidschanerin","Amerikaner", "Amerikanerin"]
# apply labelling to test set
df_test["label"]=df_test.text.apply(label_origin)
df_test[df_test["label"]=="Nationality"][["text"]].head()